In [44]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from tqdm.notebook import tqdm

In [2]:
from mlagents_envs.environment import UnityEnvironment
from gym_unity.envs import UnityToGymWrapper

In [4]:
unity_env = UnityEnvironment(base_port=5004)
env = UnityToGymWrapper(unity_env, 0)

2020-12-06 16:54:32 INFO [environment.py:205] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
2020-12-06 16:54:55 WARNING [environment.py:104] WARNING: The communication API versions between Unity and python differ at the minor version level. Python API: 1.2.0, Unity API: 1.0.
This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases
2020-12-06 16:54:55 INFO [environment.py:271] Connected new brain:
My Behavior?team=0
2020-12-06 16:54:55 WARNING [__init__.py:92] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.


In [17]:
state = env.reset()
state

array([1.        , 1.        , 1.        , 1.        , 0.        ,
       0.95693964, 0.        , 0.7392506 , 0.        , 0.5879624 ,
       0.        , 0.50567734, 0.        , 0.2557238 , 0.        ,
       0.39334723, 0.        , 0.24194278, 0.        , 0.3300575 ,
       0.        , 0.26251084, 0.        , 0.291953  , 0.        ,
       0.2322044 , 0.        , 0.26906538, 0.        , 0.21400076,
       0.7071068 , 0.7071068 , 0.7709619 , 0.        ], dtype=float32)

In [6]:
def get_il_state(state):
    idx = [1,3,5,7,9,11,13,15,17,19,21,23,25,27,29,30,31,32,33]
    return state[idx]

In [18]:
state = get_il_state(state)
len(state)

19

Process the data

In [23]:
import pandas as pd

C:\Users\30797\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [24]:
records = pd.read_csv('record_data(Capsule (6)_dest).csv') # generated by playing the game.

In [27]:
records.head()

,SD0,SD1,SD2,SD3,SD4,SD5,SD6,SD7,SD8,SD9,...,SD11,SD12,SD13,SD14,RotY,RotW,DistRatio,PathAvailable,steer,acceleration
0,1.0,1.0,0.95694,0.739251,0.587962,0.505677,0.255724,0.393347,0.241943,0.330058,...,0.291953,0.232204,0.269065,0.214001,0.707107,0.707107,0.770962,0,0.0,0.0
1,1.0,1.0,0.95694,0.739251,0.587962,0.505677,0.255724,0.393347,0.241943,0.330058,...,0.291953,0.232204,0.041873,0.214001,0.707107,0.707107,0.770962,0,0.0,0.0
2,1.0,1.0,0.95694,0.739251,0.587962,0.505677,0.255724,0.393347,0.241943,0.054202,...,0.046231,0.232204,0.044347,0.214001,0.707107,0.707107,0.770962,0,0.0,0.0
3,1.0,1.0,0.95694,0.739251,0.587962,0.505677,0.255724,0.072779,0.241943,0.068793,...,0.072484,0.232204,0.269065,0.214001,0.707107,0.707107,0.770962,0,0.0,0.0
4,1.0,1.0,0.95694,0.739251,0.587962,0.103741,0.255724,0.096522,0.241943,0.104358,...,0.291953,0.232204,0.269065,0.214001,0.707107,0.707107,0.770962,0,0.0,0.0


In [40]:
from sklearn.model_selection import train_test_split

Y_values = records[['steer','acceleration']]
X_values = records.drop(['steer','acceleration'], axis=1)


In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_values, Y_values, test_size=0.2, random_state=42)

In [51]:
train_target = torch.Tensor(y_train.values)
train = torch.Tensor(X_train.values)
train_tensor = TensorDataset(train, train_target) 
train_loader = DataLoader(dataset = train_tensor, batch_size = 32, shuffle = True)

In [53]:
X_test = torch.Tensor(X_test.values)
y_test = torch.Tensor(y_test.values)

In [45]:
for batch in train_loader:
    break

In [47]:
len(batch[0])

32

In [13]:
class ImitationNet(nn.Module):
    def __init__(self, input_dim, out_dim):
        super().__init__()
        self.fc1 = nn.Linear(in_features=input_dim, out_features=125)
        self.fc2 = nn.Linear(in_features=125, out_features=250)
        self.out = nn.Linear(in_features=250, out_features=out_dim)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        output = self.out(x)
        return output

In [14]:
net = ImitationNet(19,2)

In [19]:
state = torch.FloatTensor(state)
state = state.view(1,-1)

In [48]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001)

In [54]:
loss_list = []
eval_list = []
for itr in tqdm(range(1000)):
    for x_train,y_train in train_loader:
        y_pred = net(x_train)
        loss = criterion(y_pred,y_train)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         print("loss=",loss.item())
        loss_list.append(loss.item())
    
    # eval
    with torch.no_grad():
        y_eval = net(X_test)
        loss = criterion(y_eval,y_test)
        eval_list.append(loss.item())
        print("eval loss=",loss.item())

        

eval loss= 0.033688779920339584
eval loss= 0.03538091480731964
eval loss= 0.031585443764925
eval loss= 0.030058788135647774
eval loss= 0.031551606953144073
eval loss= 0.0328105203807354
eval loss= 0.027646321803331375
eval loss= 0.02795502543449402
eval loss= 0.02788143791258335
eval loss= 0.027965372428297997
eval loss= 0.030148521065711975
eval loss= 0.029092630371451378
eval loss= 0.026253363117575645
eval loss= 0.025038480758666992
eval loss= 0.028177209198474884
eval loss= 0.027142172679305077
eval loss= 0.026082145050168037
eval loss= 0.0277001541107893
eval loss= 0.027013611048460007
eval loss= 0.024624764919281006
eval loss= 0.024625563994050026
eval loss= 0.027589380741119385
eval loss= 0.024784771725535393
eval loss= 0.030800141394138336
eval loss= 0.024102680385112762
eval loss= 0.023988069966435432
eval loss= 0.02598346211016178
eval loss= 0.025401776656508446
eval loss= 0.023869533091783524
eval loss= 0.02302471548318863
eval loss= 0.022819818928837776
eval loss= 0.0246276

eval loss= 0.010948624461889267
eval loss= 0.009503958746790886
eval loss= 0.010779800824820995
eval loss= 0.00996007863432169
eval loss= 0.009984192438423634
eval loss= 0.009712278842926025
eval loss= 0.012181076221168041
eval loss= 0.009935245849192142
eval loss= 0.010146517306566238
eval loss= 0.009806795977056026
eval loss= 0.010486393235623837
eval loss= 0.010043347254395485
eval loss= 0.01025454606860876
eval loss= 0.01127922534942627
eval loss= 0.01051968615502119
eval loss= 0.010240653529763222
eval loss= 0.010639933869242668
eval loss= 0.010340355336666107
eval loss= 0.01081786397844553
eval loss= 0.009638284333050251
eval loss= 0.010130557231605053
eval loss= 0.011397991329431534
eval loss= 0.011385654099285603
eval loss= 0.010019761510193348
eval loss= 0.011214260943233967
eval loss= 0.010509091429412365
eval loss= 0.00960282701998949
eval loss= 0.010543416254222393
eval loss= 0.009784568101167679
eval loss= 0.010527566075325012
eval loss= 0.011418258771300316
eval loss= 0.0

eval loss= 0.007470457814633846
eval loss= 0.007290591020137072
eval loss= 0.008262787945568562
eval loss= 0.007317052222788334
eval loss= 0.008122487924993038
eval loss= 0.008265696465969086
eval loss= 0.009708702564239502
eval loss= 0.008694170974195004
eval loss= 0.008096801117062569
eval loss= 0.008084801025688648
eval loss= 0.008232786320149899
eval loss= 0.008145155385136604
eval loss= 0.00773716950789094
eval loss= 0.008012684993445873
eval loss= 0.008124872110784054
eval loss= 0.008355654776096344
eval loss= 0.008620739914476871
eval loss= 0.008431182242929935
eval loss= 0.008575603365898132
eval loss= 0.00713769905269146
eval loss= 0.008614381775259972
eval loss= 0.008761554025113583
eval loss= 0.008036844432353973
eval loss= 0.00876785721629858
eval loss= 0.00715646892786026
eval loss= 0.007695450913161039
eval loss= 0.0065340278670191765
eval loss= 0.007692412473261356
eval loss= 0.008527456782758236
eval loss= 0.00791481975466013
eval loss= 0.007741223089396954
eval loss= 0

eval loss= 0.006606257054954767
eval loss= 0.006426597014069557
eval loss= 0.007689421996474266
eval loss= 0.008069857954978943
eval loss= 0.006801928393542767
eval loss= 0.0062271347269415855
eval loss= 0.006620336789637804
eval loss= 0.0060026138089597225
eval loss= 0.006208184640854597
eval loss= 0.006975846365094185
eval loss= 0.006986622698605061
eval loss= 0.007281238213181496
eval loss= 0.0061776041984558105
eval loss= 0.006018452811986208
eval loss= 0.007003475911915302
eval loss= 0.007320334669202566
eval loss= 0.006629183888435364
eval loss= 0.006585066206753254
eval loss= 0.006400060839951038
eval loss= 0.007765328511595726
eval loss= 0.006180803757160902
eval loss= 0.005959798581898212
eval loss= 0.0076026334427297115
eval loss= 0.006561713758856058
eval loss= 0.006319534499198198
eval loss= 0.006120660807937384
eval loss= 0.006397625897079706
eval loss= 0.005976167507469654
eval loss= 0.006472940556704998
eval loss= 0.008209671825170517
eval loss= 0.007543257903307676
eval

In [86]:
def get_action(state):
    if len(state) == 34:
        state = get_il_state(state)
    with torch.no_grad():
        state = torch.Tensor(state).view(1,-1)
        action = net(state)
    return action.numpy()

def il_eval():
    state = env.reset()
    score = 0
    max_t = 10000
    for t in range(max_t):
        action = get_action(state)
        next_state, reward, done, _ = env.step(action)
        next_state = get_il_state(next_state)
        state = next_state
        score += reward
        if done:
            break 
    print("score=", score)
    
env = UnityToGymWrapper(UnityEnvironment(base_port=5004), 0)
il_eval()

2020-12-06 18:37:17 INFO [environment.py:205] Listening on port 5004. Start training by pressing the Play button in the Unity Editor.
2020-12-06 18:37:22 WARNING [environment.py:104] WARNING: The communication API versions between Unity and python differ at the minor version level. Python API: 1.2.0, Unity API: 1.0.
This means that some features may not work unless you upgrade the package with the lower version.Please find the versions that work best together from our release page.
https://github.com/Unity-Technologies/ml-agents/releases
2020-12-06 18:37:22 INFO [environment.py:271] Connected new brain:
My Behavior?team=0
2020-12-06 18:37:22 WARNING [__init__.py:92] The environment contains multiple observations. You must define allow_multiple_obs=True to receive them all. Otherwise, only the first visual observation (or vector observation ifthere are no visual observations) will be provided in the observation.


KeyboardInterrupt: 